<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

In [1]:
# Bootstrap the directory
%run "F:\\onedrive\\Desktop\\DATA SCIENCE\\Project 1\\Cross-Market Analysis\\bootstrap.py"

In [2]:
import requests
import pandas as pd
from src.config.settings import (Coingecko_Price_Interval, Coingecko_Price_Endpoint, Top_N_Coins, Coingecko_Base_URL, Coingecko_VS_Currency, Coingecko_Price_Days)
from src.validation.crypto_checks import get_top_coins

id                    0
symbol                0
name                  0
current_price         0
market_cap            0
market_cap_rank       0
total_volume          0
circulating_supply    0
total_supply          2
ath                   0
atl                   0
date                  0
dtype: int64
id                 0
symbol             0
name               0
market_cap         0
market_cap_rank    0
dtype: int64
id                    0
symbol                0
name                  0
current_price         0
market_cap            0
market_cap_rank       0
total_volume          0
circulating_supply    0
total_supply          0
ath                   0
atl                   0
date                  0
dtype: int64
0
            id symbol      name  current_price     market_cap  \
0      bitcoin    btc   Bitcoin   88692.000000  1773542722104   
1     ethereum    eth  Ethereum    2930.660000   354173986486   
2       tether   usdt    Tether       0.998506   186620060068   
3  binancecoin    

In [3]:
df_meta = pd.read_csv("F:\\onedrive\\Desktop\\DATA SCIENCE\\Project 1\\Cross-Market Analysis\\Cross-Market Data\\cleaned\\crypto\\coin_list_meta.csv")

In [4]:
top_coins = get_top_coins(df_meta, Top_N_Coins)

# Loop API calls

In [5]:
# Function to get current prices from coingecko, nested in the dictionary
def fetch_prices(top_coins):

    coin_id = top_coins
    hist_prices = {}

    for i in coin_id:
        url = f"{Coingecko_Base_URL}/coins/{i}/market_chart"
        parameters = {
                "vs_currency" : Coingecko_VS_Currency,
                "days" : Coingecko_Price_Days,
                "interval" : Coingecko_Price_Interval
                }
        response = requests.get(url, params = parameters)
        data= response.json()
        hist_prices[i] = data
    return hist_prices

hist_prices = fetch_prices(top_coins)
list(hist_prices.keys())

['bitcoin', 'ethereum', 'tether', 'binancecoin', 'ripple']

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [6]:
# Convert the nested dictionary into prices df
def extract_prices(fetch_prices):
    df_list = []

    for i, j in fetch_prices.items():

         # Defensive checks(running pipeline)
        if not isinstance(j, dict):
            print(f"⚠️ Skipping {i}: response not dict")
            continue

        if "prices" not in j:
            print(f"⚠️ Skipping {i}: 'prices' key missing")
            continue

        if not j["prices"]:
            print(f"⚠️ Skipping {i}: empty prices")
            continue

        # Extracting prices
        df_coin = pd.DataFrame(j['prices'], columns = ['timestamp', 'price'])
        # Naming the coins
        df_coin['coin_id'] = i
        df_list.append(df_coin[['coin_id', 'timestamp', 'price']])

    if not df_list:
        raise ValueError("❌ No valid price data extracted")

    # Convert all coin iterations into one single df.
    final_df = pd.concat(df_list, ignore_index = True)
    return final_df

hist_prices = fetch_prices(top_coins)
price_df = extract_prices(hist_prices)


⚠️ Skipping bitcoin: 'prices' key missing
⚠️ Skipping ethereum: 'prices' key missing
⚠️ Skipping tether: 'prices' key missing
⚠️ Skipping binancecoin: 'prices' key missing
⚠️ Skipping ripple: 'prices' key missing


ValueError: ❌ No valid price data extracted

In [ ]:
price_df.head()

In [ ]:
price_df[1]

In [ ]:
price_df.info()

In [ ]:
price_df.isnull().sum()

In [ ]:
# Export the raw df
price_df.to_csv("F:\\onedrive\\Desktop\\DATA SCIENCE\\Project 1\\Cross-Market Analysis\\Cross-Market Data\\raw\\crypto\\crypto_prices.csv")